In [64]:
import datetime
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score 
from sklearn.preprocessing import LabelEncoder

In [65]:
train_data = pd.read_csv('/projectnb/ds340/students/jmijares/CS506/train.csv')
train_data.head()

,id,trans_num,trans_date,trans_time,unix_time,category,amt,cc_num,first,last,...,zip,lat,long,city_pop,job,dob,merchant,merch_lat,merch_long,is_fraud
0,308467,26ad750c2ff71f32631b58913582d70a,2024-01-10,06:49:39,1704887379,misc_pos,188.38,676355457570,Andrea,Johnson,...,62220,38.5127,-89.9847,95666,Accounting technician,1983-05-26,fraud_Turcotte-Halvorson,39.268874,-89.273447,0
1,261578,fea9c1efe3f2b97f27ad0ab5409ec861,2024-01-06,02:37:50,1704526670,grocery_pos,102.63,377178373574671,Rhonda,Chavez,...,21784,39.4567,-76.9696,37941,"Designer, graphic",1976-12-03,fraud_Schamberger-O'Keefe,39.961495,-76.707640,0
2,341,2ae350b982be840f3666273e0c2f3a05,2024-01-18,21:40:21,1705632021,entertainment,1.62,3599292013370451,Stephen,Khan,...,49735,45.0125,-84.6723,19515,Careers information officer,1999-08-24,"fraud_Nicolas, Hills and McGlynn",44.393561,-85.342323,0
3,1147639,bbdd8adfc0a34ed0e817f809193c85c0,2024-01-21,16:20:15,1705872015,health_fitness,5.64,3594292572430345,Justin,Reilly,...,44256,41.1404,-81.8584,62039,"Merchandiser, retail",1930-02-24,fraud_Cormier LLC,40.283764,-81.639361,0
4,314152,fc7756004dc2a9bc450eb894a670b804,2024-01-21,19:36:26,1705883786,health_fitness,97.09,4867547663675548,Alice,Duarte,...,91501,34.1862,-118.3009,106841,Prison officer,1951-10-15,fraud_Kulas Group,35.149704,-118.087440,0


In [66]:
train_data.drop(columns=['cc_num','first', 'last', 'street', 'city', 
                         'state', 'zip', 'dob', 'trans_num','trans_date', 'trans_time'],inplace=True)

encoder = LabelEncoder()
train_data["merchant"] = encoder.fit_transform(train_data["merchant"])
train_data["category"] = encoder.fit_transform(train_data["category"])
train_data["gender"] = encoder.fit_transform(train_data["gender"])
train_data["job"] = encoder.fit_transform(train_data["job"])

#Drop all rows that contain missing values 
train_data.dropna(ignore_index=True)

train_data.head()

,id,unix_time,category,amt,gender,lat,long,city_pop,job,merchant,merch_lat,merch_long,is_fraud
0,308467,1704887379,9,188.38,0,38.5127,-89.9847,95666,6,638,39.268874,-89.273447,0
1,261578,1704526670,4,102.63,0,39.4567,-76.9696,37941,160,550,39.961495,-76.707640,0
2,341,1705632021,0,1.62,1,45.0125,-84.6723,19515,80,450,44.393561,-85.342323,0
3,1147639,1705872015,5,5.64,1,41.1404,-81.8584,62039,377,105,40.283764,-81.639361,0
4,314152,1705883786,5,97.09,0,34.1862,-118.3009,106841,451,350,35.149704,-118.087440,0


## Final Model Training

In [67]:
X = train_data.drop(columns=["is_fraud"], inplace = False)
Y = train_data["is_fraud"]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [24]:
pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [44]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# Define the parameter grid
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Initialize the XGBoost model
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,  # 3-fold cross-validation
    verbose=1,
    n_jobs=-1  # Use all processors
)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Best parameters and score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Score: {grid_search.best_score_:.4f}")


Fitting 3 folds for each of 243 candidates, totalling 729 fits


/usr4/ds340/jmijares/.local/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [22:56:24] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr4/ds340/jmijares/.local/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [22:56:24] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr4/ds340/jmijares/.local/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [22:56:24] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr4/ds340/jmijares/.local/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [22:56:24] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr4/ds340/jmijares/.local/lib/python3.11/site-packages/xgboost/core.py:158: UserWa

Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200, 'subsample': 1.0}
Best Score: 0.9902


In [26]:
model.score(X, Y)

0.9924440859663936

In [45]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

# Define the best parameters
best_params = {
    'colsample_bytree': 1.0,
    'learning_rate': 0.2,
    'max_depth': 7,
    'n_estimators': 200,
    'subsample': 1.0
}

# Create and train the XGBoost model with the best parameters
xgb_model = xgb.XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


/usr4/ds340/jmijares/.local/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [23:00:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.9910

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     65592
           1       0.98      0.94      0.96      8549

    accuracy                           0.99     74141
   macro avg       0.99      0.97      0.98     74141
weighted avg       0.99      0.99      0.99     74141



## Test

In [54]:
test_data = pd.read_csv("/projectnb/ds340/students/jmijares/CS506/test.csv")
test_data.head()

,id,trans_num,trans_date,trans_time,unix_time,category,amt,cc_num,first,last,...,state,zip,lat,long,city_pop,job,dob,merchant,merch_lat,merch_long
0,52329,2e6b34f2047158280fd5b50cb5249fcc,2024-01-27,13:17:44,1706379464,kids_pets,13.00,30184874050384,Edward,Mueller,...,NY,11230,40.6225,-73.9650,2504700,Leisure centre manager,1955-12-17,"fraud_Lowe, Dietrich and Erdman",40.707029,-74.027386
1,92215,5e4c36e1e6f1838f0afe1ed83d42d48e,2024-01-31,21:12:51,1706753571,travel,25.64,3560293989785735,Ryan,Reeves,...,CA,92504,33.9315,-117.4119,419138,Mudlogger,1940-06-22,fraud_Johnston-Casper,34.344545,-117.348319
2,107070,de58b3413be0b956c261b8e756006b5d,2024-01-24,23:06:59,1706155619,home,99.48,213175392060268,Gregory,Graham,...,KY,42629,36.9680,-85.0968,4953,"Engineer, automotive",1993-03-18,"fraud_Gutmann, McLaughlin and Wiza",37.493843,-85.224136
3,117508,63e5e8954b6954121fb9395b8fb87ec3,2024-01-15,14:42:37,1705347757,grocery_pos,972.26,2720994415033785,Jessica,Carey,...,TX,75571,33.1808,-94.7639,2846,Geophysicist/field seismologist,1958-06-30,"fraud_Schoen, Kuphal and Nitzsche",32.238558,-94.085343
4,525132,f0acdc291ca35b61a873060e419b20a5,2024-01-30,22:02:41,1706670161,personal_care,165.04,639070744995,Corey,Rogers,...,NJ,7022,40.8170,-74.0000,13835,"Accountant, chartered public finance",1972-04-13,fraud_Sporer-Keebler,40.957527,-73.328707


In [55]:
test_data.drop(columns=['cc_num','first', 'last', 'street', 'city', 'state', 'zip',
                        'dob', 'trans_num','trans_date', 'trans_time'],inplace=True)
encoder = LabelEncoder()
test_data["merchant"] = encoder.fit_transform(test_data["merchant"])
test_data["category"] = encoder.fit_transform(test_data["category"])
test_data["gender"] = encoder.fit_transform(test_data["gender"])
test_data["job"] = encoder.fit_transform(test_data["job"])

test_data.head()

,id,unix_time,category,amt,gender,lat,long,city_pop,job,merchant,merch_lat,merch_long
0,52329,1706379464,kids_pets,13.00,M,40.6225,-73.9650,2504700,Leisure centre manager,"fraud_Lowe, Dietrich and Erdman",40.707029,-74.027386
1,92215,1706753571,travel,25.64,M,33.9315,-117.4119,419138,Mudlogger,fraud_Johnston-Casper,34.344545,-117.348319
2,107070,1706155619,home,99.48,M,36.9680,-85.0968,4953,"Engineer, automotive","fraud_Gutmann, McLaughlin and Wiza",37.493843,-85.224136
3,117508,1705347757,grocery_pos,972.26,F,33.1808,-94.7639,2846,Geophysicist/field seismologist,"fraud_Schoen, Kuphal and Nitzsche",32.238558,-94.085343
4,525132,1706670161,personal_care,165.04,M,40.8170,-74.0000,13835,"Accountant, chartered public finance",fraud_Sporer-Keebler,40.957527,-73.328707


In [57]:
# Predict on the test data
predictions = model.predict(test_data)

# Add the predictions to the test data
test_data['is_fraud'] = predictions

# Display the first few rows with predictions
print(test_data.head())

       id   unix_time  category     amt  gender      lat      long  city_pop  \
0   52329  1706379464         7   13.00       1  40.6225  -73.9650   2504700   
1   92215  1706753571        13   25.64       1  33.9315 -117.4119    419138   
2  107070  1706155619         6   99.48       1  36.9680  -85.0968      4953   
3  117508  1705347757         4  972.26       0  33.1808  -94.7639      2846   
4  525132  1706670161        10  165.04       1  40.8170  -74.0000     13835   

   job  merchant  merch_lat  merch_long  is_fraud  
0  345       396  40.707029  -74.027386         0  
1  385       292  34.344545 -117.348319         0  
2  197       213  37.493843  -85.224136         0  
3  276       563  32.238558  -94.085343         1  
4    5       591  40.957527  -73.328707         0  


In [58]:
# Keep only 'id' and 'is_fraud' columns
test_data = test_data[['id', 'is_fraud']]

In [59]:
# Count the values in the "is_fraud" column
value_counts = test_data['is_fraud'].value_counts()

# Display the counts
print(value_counts)

is_fraud
0    82544
1    10132
Name: count, dtype: int64


In [52]:
# Save the modified DataFrame to a CSV file
test_data.to_csv('/projectnb/ds340/students/jmijares/CS506/predictions.csv', index=False)